In [80]:
import pandas as pd
import requests
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import time
import pymysql

In [81]:
conn = pymysql.connect(host='49.247.132.235', user='kbo_db', password='1234',
                       db='kbo_data_db', charset='utf8')

In [82]:
curs = conn.cursor(pymysql.cursors.DictCursor)

In [83]:
sql = '''
select player_id
,group_concat(player_year SEPARATOR ',') as year 
from kbo_hitter_total_status
group by  player_id;
'''

In [84]:
curs.execute(sql)

227

In [85]:
rows = curs.fetchall()

In [86]:
play_info = {}
for row in rows:
    play_info[row['player_id']] = row['year'].split(',')
len(play_info)

227

In [87]:
def get_status(bs,num,table_name,year,player_id):
        try:
            tag_name = '#contents > div.sub-content > div.player_records > \
            div:nth-child({}) > table > tbody > tr'.format(num)
            trs = bs.select(tag_name)
            for tr in trs:
                data_list = []
                tds = tr.select('td')
                for td in tds:
                    data_list.append(td.text)
                sql = '''
                    insert into {} values({},'{}','{}','{}','{}','{}','{}'
                    ,'{}','{}','{}','{}','{}','{}','{}');
                    '''.format(table_name,player_id,year,data_list[0],data_list[1]
                               ,data_list[2],data_list[3],data_list[4],data_list[5]
                               ,data_list[6],data_list[7],data_list[8],data_list[9]
                               ,data_list[10],data_list[11])
                curs.execute(sql)
        except:
            print('{}번 선수 {} 년도 에러발생'.format(player_id,year))
            pass

In [88]:
driver = webdriver.Chrome('E:/chromedriver_win32/chromedriver.exe')
# driver = webdriver.Chrome('E:/BigData/BigData/driver/chromedriver.exe')
#선수와 선수의 출전 년도로 반복문 실행
tag_num = {
        5: 'kbo_hitter_situation_baseman'
        ,7: 'kbo_hitter_situation_ball_count'
        ,9: 'kbo_hitter_situation_inning'
        ,11: 'kbo_hitter_situation_hitter_num'
        ,13: 'kbo_hitter_situation_pitcher_type'
        ,15: 'kbo_hitter_situation_out_count_type'
        }

print('='*30,'start','='*30)
start_time =  time.time()
count=0
for player_id,years in play_info.items():
    count+=1
#     print(count)
#     if count >= 5:
#         break
    url = 'https://www.koreabaseball.com/Record/Player/HitterDetail/Situation.aspx?playerId={}'.format(player_id)
    driver.get(url)
    time.sleep(1)
#     해당 선수의 년도별 페이지 반복문 실행
    try:
        for year in years:
            select = Select(driver.find_element_by_id('cphContents_cphContents_cphContents_ddlYear'))
            if int(year) >= 2011:
                select.select_by_value(year)
                time.sleep(1)
                html = driver.page_source
                bs = BeautifulSoup(html,'html.parser')
        #         해당 상황별 데이터를 반복문으로 데이터를 가져와서 인서트 쿼리 실행
                for num,table_name in tag_num.items():
                    get_status(bs,num,table_name,year,player_id)
                time.sleep(1)
    except:
        print('{}선수 {} 년도 에러발성'.foramt(player_id))
        pass
            
end_time = time.time()
total_time = end_time - start_time
print('실행 완료')
print(total_time)

============================== start ==============================
60339번 선수 2012 년도 에러발생
60339번 선수 2012 년도 에러발생
60339번 선수 2012 년도 에러발생
60339번 선수 2012 년도 에러발생
60339번 선수 2012 년도 에러발생
60339번 선수 2012 년도 에러발생
60339번 선수 2019 년도 에러발생
60339번 선수 2019 년도 에러발생
60339번 선수 2019 년도 에러발생
60339번 선수 2019 년도 에러발생
60339번 선수 2019 년도 에러발생
60339번 선수 2019 년도 에러발생
61268번 선수 2015 년도 에러발생
61268번 선수 2015 년도 에러발생
61268번 선수 2015 년도 에러발생
61268번 선수 2015 년도 에러발생
61268번 선수 2015 년도 에러발생
61268번 선수 2015 년도 에러발생
61295번 선수 2016 년도 에러발생
61295번 선수 2016 년도 에러발생
61295번 선수 2016 년도 에러발생
61295번 선수 2016 년도 에러발생
61295번 선수 2016 년도 에러발생
61295번 선수 2016 년도 에러발생
61365번 선수 2011 년도 에러발생
61365번 선수 2011 년도 에러발생
61365번 선수 2011 년도 에러발생
61365번 선수 2011 년도 에러발생
61365번 선수 2011 년도 에러발생
61365번 선수 2011 년도 에러발생
61895번 선수 2015 년도 에러발생
61895번 선수 2015 년도 에러발생
61895번 선수 2015 년도 에러발생
61895번 선수 2015 년도 에러발생
61895번 선수 2015 년도 에러발생
61895번 선수 2015 년도 에러발생
61895번 선수 2017 년도 에러발생
61895번 선수 2017 년도 에러발생
61895번 선수 2017 년도 에러발생
61895번 선수 2017 년도 에러발생
61895번 선수 20

In [90]:
conn.commit()

In [91]:
conn.close()